# Showcasing Large Prompt Lengths with Claude on Amazon Bedrock

## Setup the environment

In [1]:
%pip install PyPDF2
%pip install --upgrade boto3

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import json
import PyPDF2

In [3]:
bedrock = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

## Read in a book into memory for summarization

In [4]:
# Open the PDF file in binary mode
with open('The-Great-Gatsby.pdf', 'rb') as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Initialize an empty string to store the text
    text = ""

    # Iterate through each page in the PDF
    for page in pdf_reader.pages:
        # Extract text from the page
        page_text = page.extract_text()

        # Append the text from this page to the overall text
        text += page_text

# Now, 'text' contains the text extracted from the PDF
book = text.replace('\n', ' ')


## Check the number of words in the book

In [5]:
words = len(text.split(' '))
print(f'Our book has {words} words which equates roughly to {words*1.333:.0f} tokens')

Our book has 48064 words which equates roughly to 64069 tokens


## Summarize the book with a single prompt with Claude

In [6]:
prompt =f'''Human: Summarize the plot of this book.

<book>
{book}
</book>

Assistant:'''

body = json.dumps({
    "prompt": prompt,
    "max_tokens_to_sample": 1000,
    "temperature": 0.0,
    "top_p": 0.9,
})

response = bedrock.invoke_model(
    body=body, modelId='anthropic.claude-instant-v1', accept='application/json', contentType='application/json'
)

response_body = json.loads(response.get('body').read())
print(response_body.get('completion'))

 Here are the key points I picked up from reading The Great Gatsby:

- The story is narrated by Nick Carraway, a Yale graduate who moves to New York to learn about bond business. He rents a house in West Egg next to Jay Gatsby. 

- Gatsby is a mysterious man who throws lavish parties every weekend but no one knows where he comes from or what he does. 

- Nick's cousin Daisy Buchanan is married to Tom Buchanan, a wealthy man. Daisy and Gatsby had a romantic relationship years ago before she married Tom.

- Gatsby is secretly in love with Daisy and is trying to win her back. He buys a mansion across the bay from her house, hoping to attract her attention. 

- One day Nick invites Daisy to tea at Gatsby's house. This is the first time they see each other in years. It's clear they still have strong feelings for each other. 

- Tom grows suspicious of Daisy and Gatsby's relationship. At a confrontation at Tom and Daisy's house, Gatsby admits he's in love with Daisy. 

- Daisy's husband Geor